In [90]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install negspacy
!{sys.executable} -m pip install spacy_stanza 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:
import spacy
import stanza
import spacy_stanza

In [92]:
nlp = spacy_stanza.load_pipeline('es', package='ancora') #spanish

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | ancora  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [93]:
import json
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [94]:
with open("negacio_uab_revised_version.json","r") as f:
    data = json.load(f)

In [95]:
texts = []
start = []
end = []
labels = []

for i in range(len(data)):
    if i != 183:
        texts.append(data[i]['data']['text'])
        start.append(data[i]['predictions'][0]['result'][0]['value']['start'])
        end.append(data[i]['predictions'][0]['result'][0]['value']['end'])
        labels.append(data[i]['predictions'][0]['result'][0]['value']['labels'][0])

In [96]:
# Define the rules
rules = []
negation_words = []
uncertainty_words = []

for text, start_ind, end_ind, label in zip(texts, start, end, labels):
  if label == 'NEG':
    if text[start_ind:end_ind-1] not in negation_words:
      negation_words.append(text[start_ind:end_ind-1])
  elif label == 'UNC':
    if text[start_ind:end_ind-1] not in uncertainty_words:
      uncertainty_words.append(text[start_ind:end_ind-1])


In [97]:
for neg_word in negation_words:
  rules.append({'keyword': neg_word, 'category': 'NEG'})

for unc_word in uncertainty_words:
  rules.append({'keyword': unc_word, 'category': 'UNC'})

In [98]:
rules

[{'keyword': 'sin', 'category': 'NEG'},
 {'keyword': 'no', 'category': 'NEG'},
 {'keyword': 'ex-fumador', 'category': 'NEG'},
 {'keyword': 'negativo', 'category': 'NEG'},
 {'keyword': 'niega', 'category': 'NEG'},
 {'keyword': 'neg', 'category': 'NEG'},
 {'keyword': 'nega', 'category': 'NEG'},
 {'keyword': 'negativa', 'category': 'NEG'},
 {'keyword': 'inestabilidad', 'category': 'NEG'},
 {'keyword': 'niegan', 'category': 'NEG'},
 {'keyword': 'desorientacion', 'category': 'NEG'},
 {'keyword': 'sense', 'category': 'NEG'},
 {'keyword': 'ex fumador', 'category': 'NEG'},
 {'keyword': 'probable', 'category': 'UNC'},
 {'keyword': 'sugestivo de', 'category': 'UNC'},
 {'keyword': 'sospecha de', 'category': 'UNC'},
 {'keyword': 'dudoso', 'category': 'UNC'},
 {'keyword': 'sospecha', 'category': 'UNC'},
 {'keyword': 'compatible con', 'category': 'UNC'},
 {'keyword': 'posible', 'category': 'UNC'},
 {'keyword': 'probablemente', 'category': 'UNC'},
 {'keyword': 'sospechosa de', 'category': 'UNC'}]

In [174]:
def clean(tokens):
  # Start text at word 'informe'
  try:
    start_index = tokens.index('informe')
    tokens = tokens[start_index:]
  except ValueError:
    print('Word "informe" not found')

  # Remove punctuations and give PoS tags
  tokens = [token.lower() for token in tokens if token not in [',', '.', '(', ')', '-', '*', ':', '%', '/']]

  return tokens

In [100]:
# Looks for negation and uncertainty words, if not, categorize as unknown word
def categorize(text):
    for rule in rules:
        if text.lower() == rule['keyword']:
            return rule['category']
    return 'unknown'

In [169]:
# Stores in a list a negation or uncertainty word, and its scope (formed of the following two words)
def obtain_scopes(tokens, lis):
  scopes = {}

  for i in range(len(tokens)):
    if categorize(tokens[i]) != 'unknown':
      scope = []
      scope.append(tokens[i+1])
      for j in range(i+2, i+5):
        if tokens[j] in lis:
          scope.append(tokens[j])
        else:
          break
      print(f"{tokens[i]}: {' '.join(scope)}")
      scope = ' '.join(scope)
      scopes[tokens[i]] = scope

      return scopes

In [183]:
# Obtain a negation or uncertainty word with its scope, by looking for a pattern of NOUN + ADJ,
# and returns a dictionary with the whole sentence from the word until the pattern.

def obtain_scopes2(pos_tags, rules):
  dict_scopes = {}
  bol = False

  for ind, (word, tag) in enumerate(pos_tags.items()):
    scope = []
    word = word.lower()
    if word in rules:
      for i in range(ind, len(pos_tags.keys())):
        for j in range(ind+1, len(pos_tags.keys())):
          if (list(pos_tags.values())[j] == 'NOUN') and (list(pos_tags.values())[j+1] == 'ADJ'):
            scope.append(' '.join(list(pos_tags.keys())[i+1:j+2]))
            dict_scopes[word] = scope
            bol = True
            break
        if bol == True:
          break

  return dict_scopes

In [103]:
def obtain_tags(text):
  tags = {}

  doc = nlp(text)
  for token in doc:
    word, pos_tag = token.text, token.pos_
    tags[word] = pos_tag
  
  return tags

In [104]:
train_example = """El paciente tiene dolencias pero no hay presencia de alergias medicamentosas en él.
Sin incidencias postquirúrgicas que hayan dañado el tejido cerebral. No muestra síntomas de cardiopatía congénita.
Tenemos sospecha de una enfermedad patológica con posible afectación en la válvula tricúspide. 
El paciente está sin previos registros médicos."""

In [113]:
test_example = ['no hay presencia de alergias medicamentosas', 'Sin incidencias postquirúrgicas', 'No muestra síntomas de cardiopatía congénita', 'sospecha de una enfermedad patológica', 'posible afectación en la válvula tricuspide', 'sin previos registros médicos']

In [160]:
# With nltk and train set
scopes_nltk = []
possible_words = []

# Tokenize and give PoS tags
tokenized_words = nltk.word_tokenize(train_example)
tags = nltk.pos_tag(tokenized_words)

# Make a list of possible nouns (words that are labeled as nouns or foreign words discarding
# possible conjunctions or prepositions)
for word, pos_tag in tags:
  if ((pos_tag == 'NN') or (pos_tag == 'NNS') or (pos_tag == 'FW') or (pos_tag == 'VB') or (pos_tag == 'JJ')) and (re.match(r'\b\w+\b', word)):
    if word not in possible_words:
      possible_words.append(word)

# Localize the negation and uncertainty words and print their scopes (the following two words)
scopes_nltk = obtain_scopes(clean(tokenized_words), possible_words)

no: hay presencia de alergias


In [156]:
scopes_nltk

{'no': 'hay presencia de alergias'}

In [116]:
# With spacy and train set
neg_unc_words = [rule['keyword'] for rule in rules]
pos_tags = obtain_tags(train_example)
scopes_spacy = obtain_scopes2(pos_tags, neg_unc_words)
scopes_spacy

{'no': ['muestra síntomas cardiopatía congénita'],
 'sin': ['previos registros médicos'],
 'sospecha': ['una enfermedad patológica'],
 'posible': ['afectación la válvula tricúspide']}

In [139]:
# Display the accuracy
correct = 0
accuracy = 0

for key, value in scopes_nltk.items():
  for sentence in test_example:
    tokens_test = nltk.word_tokenize(sentence)
    tokens_train = nltk.word_tokenize(value[0])
    if tokens_test[-1] == tokens_train[-1]:
      correct += 1

accuracy = (correct / len(test_example))*100
print(f'Accuracy with nltk library: {accuracy}%')


accuracy = 0
correct = 0
for key, value in scopes_spacy.items():
  for sentence in test_example:
    tokens_test = nltk.word_tokenize(sentence)
    tokens_train = nltk.word_tokenize(value[0])
    if tokens_test[-1] == tokens_train[-1]:
      correct += 1

accuracy = (correct / len(test_example)) * 100
print(f'Accuracy with spacy library: {accuracy}%')

Accuracy with nltk library: 0.0%
Accuracy with spacy library: 50.0%


In [175]:
# With nltk and medical diagnosis
scopes_nltk = []
possible_words = []

# for i, text in enumerate(texts):
#   if i > 5:
#     break
#   else:
for text in texts:
    # Tokenize and give PoS tags
    tokenized_words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(clean(tokenized_words))

    # Make a list of possible nouns (words that are labeled as nouns or foreign words discarding
    # possible conjunctions or prepositions)
    for word, pos_tag in tags:
      if ((pos_tag == 'NN') or (pos_tag == 'NNS') or (pos_tag == 'FW') or (pos_tag == 'VB')) and (len(word) > 3) and (re.match(r'\b\w+\b', word)):
        if word not in possible_words:
          possible_words.append(word)

  
    # Localize the negation and uncertainty words and print their scopes (the following two words)
    dict_scopes_nltk = obtain_scopes(clean(tokenized_words), possible_words)

sin: hijos tiene
no: amc
no: preciso intervencion quirurgica
no: alergias medicamentosas conocidas
sin: habitos toxicos
no: alergias medicamentosas conocidas antecedentes
no: amc
probable: etiologia inmune
sense: al·lergies
no: alergias medicamentosas conocidas
no: alergias medicamentosas conocidas
no: habitos toxicos antecedentes
ex-fumador: de
no: alergias medicamentosas conocidas intervenciones
negativo: descartada
no: tolero dosis
no: alergias medicamentosas conocidas
niega: alergias medicamentosas conocidas
no: habitos toxicos
no: alergias medicamentosas conocidas
no: alergias medicamentosas conocidas
sin: alergias medicamentosas conocidas calendario
no: se
neg: aro
no: alergias medicamentosas conocidas fumador
no: confirmada
sense: complicacions altres tipus d'hiperlipidemia
no: medicacion habitual proces
no: alergias medicamentosas conocidas claudicacion
no: amc neutropenia
sin: alergias medicamentosas conocidas
sin: alergias medicamentosas conocidas
sin: alergias medicamentosas

In [176]:
dict_scopes_nltk

{'sin': 'incidencias postquirurgicas buena evolucion'}

In [184]:
# With spacy and medical diagnosis
dict_scopes_spacy = {}
for i, text in enumerate(texts):
  if i > 10:
    break
  else:
    pos_tags = obtain_tags(text)
    scopes_spacy = obtain_scopes2(pos_tags, neg_unc_words)
    dict_scopes_spacy.update(scopes_spacy)

/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  doc = self._ensure_doc(text)
/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['nº', 'historia', 'clinica', ':', '**', '***', '***', 'nºepisodi', ':', '********', 'sexe', ':', 'dona', 'data', 'de', 'naixement', ':', '06.06.1938', 'edat', ':', '79', 'anys', 'procedencia', 'domicil/res', '.soc', 'servei', 'psiquiatria', 'data', "d'ingres", '10.05.2018', 'data', "d'alta", '10.05.2018', '16:46:41', 'ates', 'per', '****************', ',', '************', ';', '*************', ',', 'assumpta', 'informe', "d'alta", "d'hospitalitzacio", 'motiu', "d'ingres", 'paciente', 'de', '79', 'años', 'que', 'acude', 

In [185]:
dict_scopes_spacy

{'sin': ['incidencias intraoperatorias'],
 'no': ['alergias medicamentosas'],
 'negativo': ['coombs indirecto'],
 'probablemente': ['secundarios viriasis elevacion reactantes valora conjuntamente equipo hematologia decide realizar para valoracion realiza procedimiento sedacion incidencias posteriores'],
 'probable': ['etiologia inmune'],
 'neg': ['rub inm toxo - lues vhc vhb vih chagas ts bajo riesgo 2t 12.5 osullivan 111 3r t :hb11.8 htc 35.7% pqtaqs 256 coag coomb sneg uro hiv cultivo vaginal/rectal ecografias 1r tm 12+3w congruente crl 62.5mm tn 1.4mm 2o 11.1.18 20+5 morfologia normal'],
 'sense': ['al·lergies conegudes fisiologics hernia hiato meningitis proces actual niña años con cuadro abodmonal'],
 'negativa': ['administro dosis presentando pico 39,3ºc durante la mala tolerancia realizado sanguinea control consutan persistencia &lt;38ºc refieren sangrado ningun nivel inflamacion periungeal'],
 'niega': ['habitos toxicos']}